In [1]:
import pandas as pd
from pathlib import Path

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib

pd.options.display.max_columns = None

In [2]:
# path of the folder with .parquet
folder_path = Path("C:/Users/AULA04/Desktop/Codes/data/box_builder_dataset/feature_frame.csv")

In [9]:
feature_frame = pd.read_csv(folder_path)

In [10]:
feature_frame=feature_frame[feature_frame["user_order_seq"]>=5].reset_index()

# Columns
* variant id                            : -> Not include, inside of the model will be the caracteristics 
* product_type                          : -> One Hot encoding
* order_id                              :
* user_id                               :
* X created_at                          : are the same row, have the hour, it's innecesary -> *Delete*
* X order_date                          : There isn't any related to the time, I think it's innecesary here -> *Delete*
* ok user_order_seq                     :
* ok outcome                            :
* ok ordered_before                     :
* ok abandoned_before                   :
* ok active_snoozed                     :
* ok set_as_regular                     :
* normalised_price                      :
* discount_pct                          :
* vendor                                :
* ok global_popularity                  :
* count_adults                          :|
* count_children                        :|
* count_babies                          :|-Datos imputados
* count_pets                            :|
* people_ex_baby                        :|
* ok days_since_purchase_variant_id     :
* ok avg_days_to_buy_variant_id         :
* ok std_days_to_buy_variant_id         :
* ok days_since_purchase_product_type   :   
* ok avg_days_to_buy_product_type       :
* ok std_days_to_buy_product_type       :


### Product type

In [11]:
#Product type: need to be categorical encoding
feature_frame = pd.concat([feature_frame, pd.get_dummies(feature_frame["product_type"])],axis=1)
feature_frame = feature_frame.drop("product_type",axis=1)
feature_frame = feature_frame.drop("index",axis=1)

### Delete: 
* user_id
* variant_id
* order_date
* created_at

In [12]:
feature_frame = feature_frame.drop("variant_id",axis=1)
feature_frame = feature_frame.drop("user_id",axis=1)
feature_frame = feature_frame.drop("order_date",axis=1)
feature_frame = feature_frame.drop("created_at",axis=1)

### order_ID

In [7]:
#Count encoding
feature_frame['order_id'] = feature_frame.order_id.map(feature_frame.order_id.value_counts())

In [13]:
#ategorical encoding
feature_frame = pd.concat([feature_frame, pd.get_dummies(feature_frame["order_id"])],axis=1)
feature_frame = feature_frame.drop("order_id",axis=1)
feature_frame

,user_order_seq,outcome,ordered_before,abandoned_before,active_snoozed,set_as_regular,normalised_price,discount_pct,vendor,global_popularity,count_adults,count_children,count_babies,count_pets,people_ex_baby,days_since_purchase_variant_id,avg_days_to_buy_variant_id,std_days_to_buy_variant_id,days_since_purchase_product_type,avg_days_to_buy_product_type,std_days_to_buy_product_type,airfreshener,allpurposecleaner,babyfood12months,babyfood6months,babymilkformula,babytoiletries,bathroomlimescalecleaner,bathshowergel,beer,binbags,bodyskincare,catfood,cereal,cleaningaccessories,coffee,condimentsdressings,cookingingredientsoils,cookingsaucesmarinades,delicatesstainremover,dental,deodorant,dishwasherdetergent,dogfood,driedfruitsnutsseeds,dryingironing,fabricconditionerfreshener,facialskincare,feedingweaning,femininecare,floorcleanerpolish,foodstorage,haircare,handsoapsanitisers,healthcarevitamins,homebaking,householdsundries,jamhoneyspreads,juicesquash,kidsdental,kidssnacks,kitchenovencleaner,kitchenrolltissues,longlifemilksubstitutes,maternity,nappies,nappypants,petcare,pickledfoodolives,premixedcocktails,ricepastapulses,shavinggrooming,snacksconfectionery,softdrinksmixers,superfoodssupplements,tea,tinspackagedfoods,toiletroll,washingcapsules,washingliquidgel,washingpowder,windowglasscleaner,wipescottonwool,2808548917380,2808549900420,2809061539972,2809064751236,2809189335172,2811003142276,2811116486788,2811283374212,2812639314052,2812646883460,2813430104196,2814728831108,2814794825860,2815257837700,2815796772996,2816268370052,2818294415492,2819489431684,2819557916804,2819619356804,2819779985540,2820035543172,2820787961988,2821035032708,2821392072836,2823366312068,2823888896132,2824022491268,2824095760516,2824508113028,2824721301636,2824766685316,2824875442308,2825084829828,2825094398084,2826374840452,2826453221508,2826939138180,2827488690308,2827770364036,2828415139972,2829010010244,2829100384388,2829694500996,2830314471556,2830833811588,2830892499076,2831025406084,2831404990596,2831411183748,2831447392388,2831796306052,2831806955652,2831842672772,2831930622084,2832596336772,2832639590532,2832996827268,2833481597060,2833503584388,2833529864324,2833564860548,2833904205956,2833933893764,2834171789444,2834218811524,2835129860228,2835134316676,2835455737988,2835732496516,2835899252868,2835968655492,2836007747716,2836134559876,2836702789764,2837167243396,2838033301636,2838979018884,2839026892932,2839053697156,2839849074820,2841051332740,2842557710468,2843277852804,2843532853380,2843543863428,2843720450180,2843785986180,2843866529924,2844273606788,2845088514180,2845558636676,2845684301956,2845920723076,2846335533188,2847021006980,2847087394948,2847092473988,2848357318788,2848868237444,2850559197316,2850726183044,2851457433732,2852832903300,2852911775876,2855911555204,2856243986564,2856795963524,2856915828868,2858159308932,2859871600772,2860941803652,2861056491652,2861786759300,2861943718020,2862841888900,2862966407300,2863037382788,2863303327876,2863600042116,2863633170564,2863637856388,2863779053700,2864142221444,2864388472964,2864585769092,2864825991300,2865130078340,2865439244420,2865773969540,2866083528836,2866194055300,2866198052996,2866589892740,2866639143044,2866735710340,2866792693892,2866868355204,2867587252356,2867985154180,2868125368452,2868664860804,2869695938692,2869936521348,2870741663876,2871542644868,2871725359236,2871728308356,2871767957636,2872515526788,2872609865860,2872693194884,2873681641604,2873805766788,2874097008772,2874136723588,2874299318404,2874649542788,2874736574596,2874795982980,2875009269892,2875017166980,2875085324420,2875211677828,2875224588420,2875521335428,2875590705284,2875684520068,2876010889348,2876412133508,2876541501572,2877023060100,2877519200388,2877603119236,2877827874948,2878720540804,2878739480708,2879418892420,2879431082116,2879490588804,2879523979396,2879788712068,2880200212612,2880434372740,2880441516164,2881475543172,2883255795844,2883886940292,2884123754628,2884404248708,2884408737924,2884

### vendor

In [14]:
counts = feature_frame.vendor.value_counts()
feature_frame['vendor'] = feature_frame.vendor.map(counts)

In [15]:
feature_frame = feature_frame*1
feature_frame.columns = feature_frame.columns.astype(str)

In [16]:
scaler = MinMaxScaler()
feature_frame_normalized = scaler.fit_transform(feature_frame)
feature_frame_normalized = pd.DataFrame(feature_frame_normalized, columns=feature_frame.columns)
feature_frame_normalized

,user_order_seq,outcome,ordered_before,abandoned_before,active_snoozed,set_as_regular,normalised_price,discount_pct,vendor,global_popularity,count_adults,count_children,count_babies,count_pets,people_ex_baby,days_since_purchase_variant_id,avg_days_to_buy_variant_id,std_days_to_buy_variant_id,days_since_purchase_product_type,avg_days_to_buy_product_type,std_days_to_buy_product_type,airfreshener,allpurposecleaner,babyfood12months,babyfood6months,babymilkformula,babytoiletries,bathroomlimescalecleaner,bathshowergel,beer,binbags,bodyskincare,catfood,cereal,cleaningaccessories,coffee,condimentsdressings,cookingingredientsoils,cookingsaucesmarinades,delicatesstainremover,dental,deodorant,dishwasherdetergent,dogfood,driedfruitsnutsseeds,dryingironing,fabricconditionerfreshener,facialskincare,feedingweaning,femininecare,floorcleanerpolish,foodstorage,haircare,handsoapsanitisers,healthcarevitamins,homebaking,householdsundries,jamhoneyspreads,juicesquash,kidsdental,kidssnacks,kitchenovencleaner,kitchenrolltissues,longlifemilksubstitutes,maternity,nappies,nappypants,petcare,pickledfoodolives,premixedcocktails,ricepastapulses,shavinggrooming,snacksconfectionery,softdrinksmixers,superfoodssupplements,tea,tinspackagedfoods,toiletroll,washingcapsules,washingliquidgel,washingpowder,windowglasscleaner,wipescottonwool,2808548917380,2808549900420,2809061539972,2809064751236,2809189335172,2811003142276,2811116486788,2811283374212,2812639314052,2812646883460,2813430104196,2814728831108,2814794825860,2815257837700,2815796772996,2816268370052,2818294415492,2819489431684,2819557916804,2819619356804,2819779985540,2820035543172,2820787961988,2821035032708,2821392072836,2823366312068,2823888896132,2824022491268,2824095760516,2824508113028,2824721301636,2824766685316,2824875442308,2825084829828,2825094398084,2826374840452,2826453221508,2826939138180,2827488690308,2827770364036,2828415139972,2829010010244,2829100384388,2829694500996,2830314471556,2830833811588,2830892499076,2831025406084,2831404990596,2831411183748,2831447392388,2831796306052,2831806955652,2831842672772,2831930622084,2832596336772,2832639590532,2832996827268,2833481597060,2833503584388,2833529864324,2833564860548,2833904205956,2833933893764,2834171789444,2834218811524,2835129860228,2835134316676,2835455737988,2835732496516,2835899252868,2835968655492,2836007747716,2836134559876,2836702789764,2837167243396,2838033301636,2838979018884,2839026892932,2839053697156,2839849074820,2841051332740,2842557710468,2843277852804,2843532853380,2843543863428,2843720450180,2843785986180,2843866529924,2844273606788,2845088514180,2845558636676,2845684301956,2845920723076,2846335533188,2847021006980,2847087394948,2847092473988,2848357318788,2848868237444,2850559197316,2850726183044,2851457433732,2852832903300,2852911775876,2855911555204,2856243986564,2856795963524,2856915828868,2858159308932,2859871600772,2860941803652,2861056491652,2861786759300,2861943718020,2862841888900,2862966407300,2863037382788,2863303327876,2863600042116,2863633170564,2863637856388,2863779053700,2864142221444,2864388472964,2864585769092,2864825991300,2865130078340,2865439244420,2865773969540,2866083528836,2866194055300,2866198052996,2866589892740,2866639143044,2866735710340,2866792693892,2866868355204,2867587252356,2867985154180,2868125368452,2868664860804,2869695938692,2869936521348,2870741663876,2871542644868,2871725359236,2871728308356,2871767957636,2872515526788,2872609865860,2872693194884,2873681641604,2873805766788,2874097008772,2874136723588,2874299318404,2874649542788,2874736574596,2874795982980,2875009269892,2875017166980,2875085324420,2875211677828,2875224588420,2875521335428,2875590705284,2875684520068,2876010889348,2876412133508,2876541501572,2877023060100,2877519200388,2877603119236,2877827874948,2878720540804,2878739480708,2879418892420,2879431082116,2879490588804,2879523979396,2879788712068,2880200212612,2880434372740,2880441516164,2881475543172,2883255795844,2883886940292,2884123754628,2884404248708,2884408737924,2884

## Divide the dataset

70 - 20 - 10


In [17]:
y = feature_frame_normalized["outcome"]
x = feature_frame_normalized.drop(columns=["outcome"])

In [18]:

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, stratify=y, random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.20, stratify=y_train, random_state=42)


print(f"Train: {len(X_train)} ")
print(f"Valid: {len(X_valid)} ")
print(f"Test: {len(X_test)} ")


MemoryError: Unable to allocate 1.67 GiB for an array with shape (653, 343152) and data type float64

In [ ]:
model = LogisticRegression(class_weight="balanced", random_state=42, max_iter=500)

model.fit(X_train, y_train)

In [ ]:
y_valid_pred = model.predict(X_valid)
print("Validación:\n", classification_report(y_valid, y_valid_pred))

In [ ]:
y_test_pred = model.predict(X_test)
print("Validación:\n", classification_report(y_test, y_test_pred))

### Save the model

In [ ]:
joblib.dump(model, "modelo_logistic_regression.pkl")